# **CPT_S 315 HW3-Coding: User-User Collaborative Filtering (40 points)**
In this homework, you are required to implement user-user collaborative filtering algorithm and return top N recommendated movies.

### **Requirements:**
Please download the ``files hw_3.ipynb``, and ``ml-100k.zip`` from Canvas and complete the codes in hw_3.ipynb following the instructions in the file. You can run and debug your codes using Google Colab. After completing the tasks, save hw_3.ipynb as hw_3.py by clicking File -> Download -> Download .py. Then upload hw_3.py to Gradescope. The autograder on Gradescope will automatically grade your coding homework.

In [2]:
# We first need to upload the file "ml-latest-small.zip" downloaded from Canvas
try:
    from google.colab import files
    uploaded = files.upload()
except ImportError as e:
    pass

Saving ml-100k.zip to ml-100k.zip


# Introduction

In some cells and files you will see code blocks that look like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
raise NotImplementedError()
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

You should replace the "raise NotImplementedError()"  with your own code and leave the blocks intact, like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
y = m * x + b
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

## **When completing the notebook, please adhere to the following rules:**
 - **Do not modify any code outside of code blocks.**
 - **You can create your own customer functions.**
 - **Do not add or delete any cells from the notebook. You may add new cells to perform scatch work, but delete them before submitting.**
 - **Make sure you run all the pre-defined functiosn before testing.**
 - **Make sure at least you pass all the testing cases before submitting.**
 - **Common packages are available, for example: sklearn, scipy, pandas, numpy. Please contact the introductors if you need to use other packages**


### **You will only get credit for code that has been run!**.

### **We will not re-run your notebook -- you will get your credits automatically on Gradescope**

## **User-User Collaborative Filtering**

In [1]:
# This cell provides some useful functions for generating the utility matrix.
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import os
import pandas as pd
import numpy as np
import zipfile
import urllib.request
import sys

def unzip(name):
    with zipfile.ZipFile(name, 'r') as data:
        data.extractall()

class ml100k:
    @staticmethod
    def load():
        name = 'ml-100k'
        unzip(f"{name}.zip")
        ratings_path = os.path.join(name, 'u.data')
        ratings = pd.read_csv(
            ratings_path,
            sep='\t',
            names=["userid", "itemid", "rating", "timestamp"],
        )
        ratings = ratings.sort_values(by=['userid', 'itemid']).reset_index(drop=True)
        ratings = ratings.drop(columns=['timestamp'])

        movies_columns = [
            'itemid', 'title', 'release date', 'video release date',
            'IMDb URL ', 'unknown', 'Action', 'Adventure', 'Animation',
            "Children's", 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
            'Film-Noir', 'Horror', 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
            'Thriller' , 'War' , 'Western' ,
        ]

        movies_path = os.path.join(name, 'u.item')
        movies = pd.read_csv(
            movies_path,
            sep='|',
            names=movies_columns,
            encoding='latin-1',
        )
        # drop non necessary columns. From the third to the last column
        todrop = list(range(2, len(movies.columns)))
        movies = movies.drop(movies.columns[todrop], axis=1)

        return ratings, movies

def ids_encoder(ratings):
    users = sorted(ratings['userid'].unique())
    items = sorted(ratings['itemid'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings.userid = uencoder.transform(ratings.userid.tolist())
    ratings.itemid = iencoder.transform(ratings.itemid.tolist())
    return ratings, uencoder, iencoder

def ratings_matrix(ratings):
    # return a rating matrix based with filling values
    return csr_matrix(pd.crosstab(ratings.userid, ratings.itemid, ratings.rating, aggfunc=sum).fillna(0).values)

# Generate utility information
def generate_utility_matrix():
    ratings, movies = ml100k.load()
    ratings, uencoder, iencoder = ids_encoder(ratings)
    return ratings

def create_model(rating_matrix, metric):
    """
    - create the nearest neighbors model with the corresponding similarity metric
    - fit the model
    """
    model = NearestNeighbors(metric=metric, n_neighbors=21, algorithm='brute')
    model.fit(rating_matrix)
    return model


def nearest_neighbors(rating_matrix):
    """
    :param rating_matrix : rating matrix of shape (nb_users, nb_items)
    :return
        - similarities : distances of the neighbors from the referenced user
        - neighbors : neighbors of the referenced user in decreasing order of similarities
    """
    rating_matrix = np.array(rating_matrix)
    if rating_matrix.shape != (943, 1682):
         raise ValueError("Rating Matrix is not of the right shape")

    model = create_model(rating_matrix, metric='cosine')
    similarities, neighbors = model.kneighbors(rating_matrix)
    return 1 - similarities[:, 1:], neighbors[:, 1:]


def find_candidate_items(userid):
    """
    Find candidate items for an active user

    :param userid : active user
    :param neighbors : users similar to the active user
    :return candidates : top 30 of candidate items
    """
    ratings_ = generate_utility_matrix()
    R_ = ratings_matrix(ratings_)
    ar = R_.toarray()
    similarities_, neighbors_ = nearest_neighbors(ar)
    user_neighbors = neighbors_[userid]
    activities = ratings.loc[ratings_.userid.isin(user_neighbors)]
    # sort items in decreasing order of frequency
    frequency = activities.groupby('itemid')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency.itemid
    active_items = ratings_.loc[ratings.userid == userid].itemid.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:30]

    return candidates

# 1. User-based Collaborative Filtering

## Idea

Let $u$ be the user for which we plan to make recommendations.

1. Find other users whose past rating behavior is similar to that of $u.$
2. Use their ratings on other items to predict what the current user will like.

## Algorithm : user-to-user collaborative filtering

The entire process of user-to-user CF algorithm is described as follow <a href="https://romisatriawahono.net/lecture/rm/survey/information%20retrieval/Bobadilla%20-%20Recommender%20Systems%20-%202013.pdf">(J. Bobadilla et al. 2013)</a>: For an active user $u$,

<ol>
    <li> First identify the set $N_u$ of $k$ most similar users. $N_u$ is the group users similar to the active user $u$. The similarity between two users $u$ and $v$ can be measured by the cosine similarity measure as follows :

\begin{equation}
 s_{u,v}=\frac{\vec{r}_u \cdot \vec{r}_v}{\|\vec{r}_u\| \ast \|\vec{r}_v\|} = \frac{\sum_{i\in I}r_{u,i}r_{v,i}}{\sqrt{\sum_{i\in I} (r_{u,i})^2}\sqrt{\sum_{i\in I} (r_{v,i})^2}}
\end{equation}

$s_{u,v}$ is the degree of similarity between users $u$ and $v$. This term is computed for all $v\in U$, where $U$ is the set of all users. $\vec{r}_v = [r_{v,1},r_{v,2},\dots,r_{v,I}]$ is the rating vector of user $v.$ $I$ is the total number of movies in the system. As experimented by <a href="https://dl.acm.org/doi/10.1145/3130348.3130372">(Herlocker et al. 1999)</a>, $k\in [20,50]$ is a reasonable starting point in many domains. So in this assignment, we select $k=20.$
    </li>
    <li> Find the set $C$ of candidate items, purchased by the group and not purchased by the active user $u$. Candidate items have to be the most frequent items purchased by the group.
    </li>
    <li>Aggregate ratings of users in $N_u$ to make predictions for user $u$ on items he has not already purchased. Several aggregation approaches are often used such as <b>average, weighted sum, ajusted weighted sum</b>. By using weighted sum, the predicted rating of user $u$ on item $i$ denoted by $\hat{r}_{u,i}$ is computed as follow :

\begin{equation}
 \hat{r}_{u,i}=\bar{r}_u + \frac{\sum_{v\in N_u}(r_{v,i}-\bar{r}_v)\cdot s_{u,v}}{\sum_{v\in N_u}|s_{u,v}|}.
\end{equation}

Ratings of similar users are weighted by the corresponding similarity with the active user. Summation are made over all the users who have rated item $i$. Subtracting the user’s mean rating $\bar{r}_v$ compensates for differences in users’ use of the rating scale as some users will tend to give higher ratings than others <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>. This prediction is made for all items $i \in C$ not purchased by user $u$.
    </li>
    <li>The Top-$N$ recommendations are obtained by choosing the $N$ items which provide most satisfaction to the user according to prediction.
    </li>
</ol>

### Step 1. Generate Utility Data

There are total $10k$ ratings in the dataset.
Runing the function
```python
ratings = generate_utility_matrix()
```
will return all the rating information $ratings.$
```python

```
For example if you would like to know the first ten ratings from the first user(userid=0), you will have
```python
       userid  itemid  rating
0           0       0       5
1           0       1       3
2           0       2       4
3           0       3       3
4           0       4       3
...       ...     ...     ...
99995     942    1066       2
99996     942    1073       4
99997     942    1187       3
99998     942    1227       3
99999     942    1329       3
```
If you would like to extract the rating from $userid = 0$ and $itemid = 4,$ you can simply run:
```python
print(ratings[(ratings.userid == 0) & (ratings.itemid == 4)]['rating'].values[0].astype(float)
```
### Note that some itemid may never be rated by an user before, the ratings matrix only contains active ratings.

### Step 2. Identify $N_u$, the set of $k=20$ users similar to an active user $u$

To find the $k$ most similar users to $u$, we use the cosine similarity and compute $w_{u,v}$ for all $v\in U$. Fortunately, libraries such as <i>scikit-learn (sklearn)</i> are very useful for such tasks :

1. Function ```similarities, neighbors = nearest_neighbors(rating_matrix, model):``` returns the k closed users ```neighbors``` for each user and the associated similarities ```similarities```. We already create this function for you.

**Your Answer is needed in the following cell:**

In [ ]:
def generate_centered_rating_matrix(ratings):
    """
    Please fill up your implementation.
    The input of the function is the original rating matrix "ratings" generated by the function "generate_utility_matrix()".
    The ratings is a pandas dataframe.

    Since we have 943 users and 1682 movies in the sytem.

    The output matrix should be a numpy array with shape 943 * 1682.
    The (i,j) element for the matrix is the centered rating from user i (with userid i) of move j (with itemid j).
    All the missing ratings are filling up with value 0.
    For the centered rating matrix, for example if the original rating matrix is:
    [[ 4 0 0 5 1 0],
     [ 5 5 4 0 0 0],
     [ 0 0 3 0 0 1]]
    Then the centered rating matrix is:
    [[ 2/3 0 0 5/3 -7/3 0],
     [ 1/3 1/3 -2/3 0 0 0],
     [ 0 0 1 0 0 -1]]
     Refer to Slide 8 - 1.

     Note that: All the missing values remain to be 0. The mean of each row is calculated among all the positive ratings (active ratings).
    """
    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    raise NotImplementedError()
    ###########################################################################
    #                            END OF YOUR CODE                             #
    ###########################################################################
    return center_rating_matrix

In [ ]:
#1. Test the centered rating matrix
ratings = generate_utility_matrix()
centered_rating = generate_centered_rating_matrix(ratings)
centered_rating = np.array(centered_rating)

r_0 =   [1.38970588 ,-0.61029412,  0.38970588, -0.61029412, -0.61029412 , 1.38970588, 0.38970588, -2.61029412 , 1.38970588 ,-0.61029412]
r_15 =  [0.67142857, 0.  ,       0.  ,       0.67142857, 0. ,        0., 0.67142857, 0.67142857 ,0.67142857 ,0. ]
r_345 = [ 0.  , 1.70984456, -0.29015544 , 0.70984456,  0. ,  0., -1.29015544 , 0.  ,  0. ,  0. ]

r_0_ = centered_rating[0][0:10]
r_15_ = centered_rating[15][0:10]
r_345_ = centered_rating[345][0:10]

r_test = [r_0,r_15,r_345]
r_pre = [r_0_,r_15_,r_345_]

err = 0
for i in range(len(r_test)):
  err += np.linalg.norm(r_test[i] - r_pre[i])

print (f" Testing error is : {err}.")
if err >= 0.01:
  print("DOUBLE CHECK YOUR CALCULATION!")
else:
  print("YOU PASSED THE TEST!")

 Testing error is : 1.1464052536472428e-08.
YOU PASSED THE TEST!


/tmp/ipykernel_1818053/739971252.py:34: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  R = csr_matrix(pd.crosstab(mean_ratings.userid, mean_ratings.itemid, mean_ratings.norm_rating, aggfunc=sum).fillna(0).values)


### Step 3. Rating prediction

Now it's time to predict what score the active user $u$ would have given to each of the top-10 candidate items.

To predict the score of $u$ on a candidate item $i$ ,we need :
1. Similarities between $u$ and all his neighbors $v \in N_u$ who rated item $i.$
2. Normalized ratings of all $v \in N_u$ on item $i$. The normalized rating of user $v$ on item $i$ is defined by $r_{v,i}-\bar{r}_v$.

**Your Answer is needed in the following cell:**

1. You need to use function ```predict``` to predict rating between user $u$ and item $i$. Recall that the prediction formula is defined as follow :

\begin{equation}
 \hat{r}_{u,i}=\bar{r}_u + \frac{\sum_{v\in N_u}(r_{v,i}-\bar{r}_v)\cdot s_{u,v}}{\sum_{v\in G_u}|s_{u,v}|}.
\end{equation}

**Your Answer is needed in the following cell:**

In [ ]:
#We can obtain the similarity matrix and neighbor matrix using the centered rating matrix.

##########################################################################
"""
centered_rating = generate_centered_rating_matrix(ratings)
similarities, neighbors = nearest_neighbors(centered_rating)
"""
##########################################################################

"""
The shape of similarities and neighbors are both 943 x 20.
It is easy to obtain the closed (most similar) k=20 users for a given user with user_neighbors = neighbors[userid]
For example user1_neighbors = neighbors[1] will return the first 20 closed users for the user with userid = 1

The returnning list contains all the similar user information (userid) in decreasing order of similarities.
Similary, similarities[userid] contains the similarities information between user and its neighbors.
"""

def predict(ratings, userid, itemid, similarities, neighbors):
    """
    predict what score userid would have given to itemid.

    :param
        - userid : user id for which we want to make prediction
        - itemid : item id on which we want to make prediction
        - similarities: a matrix contains the consin distance between user and its nearest 20 neighbors
        - neighbors: a matrix contains the most 20 closed users for each user

    :return
        - r_hat : predicted rating of user userid on item itemid
    """

    #Obtain neighbors N_u and associated similarity distance s_{u,v}
    user_similarities = similarities[userid] #list with size 20
    user_neighbors = neighbors[userid] #list with size 20

    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    #Something you need to do here
    # 1. Get mean rating of user userid
    # 2. Find users who rated item 'itemid'
    # 3. Find similar users to 'userid' who rated item 'itemid' (userids contained among the users found by last step)
    # 4. Obtain similarity distance from user_similarities

    raise NotImplementedError()

    ###########################################################################
    #                            END OF YOUR CODE                             #
    ###########################################################################

    return r_hat

In [ ]:
#2. Test the prediction for user i and item j

ratings = generate_utility_matrix()
centered_rating = generate_centered_rating_matrix(ratings)
centered_rating = np.array(centered_rating)
similarities, neighbors = nearest_neighbors(centered_rating)

test_case = [[0,422],[0,567],[0,404],[0,432],[100,257],[100,299],[850,6],[850,281]]
test_rating = [3.9486011373999577, 3.7872382581245776, 3.5241429399046993, 4.185430215904166, 3.491834899395883, 2.7749381804874993, 4.042253725491653, 3.50294950957094]
pres = []
for i in test_case:
  r_hat =  predict(ratings, i[0], i[1], similarities, neighbors)
  pres.append(r_hat)

err = np.linalg.norm(np.array(pres) - np.array(test_rating))
print (f" Testing error is : {err}.")
if err >= 0.01:
  print("DOUBLE CHECK YOUR CALCULATION!")
else:
  print("YOU PASSED THE TEST!")

### Step 4. Top-N recommendation

Function ```user2userRecommendation(userid, N)``` return the list of Top-N (N<= 20) items in decreasing order of predicted rating for user userid.

**Your Answer is needed in the following cell:**

In [ ]:
def user2userRecommendation(ratings, userid: int, N : int):
    """
    Recommend movies to user (id = userid)
    :param
        - userid : user id for which we want to make prediction
    :return
        - r_list : list of recommended itemids
    """
    # find candidate items for the active user
    # You need to recommend movies from the set of candidates
    # candidates is a list of all the ponential items that can be recommended to user (id=userid)
    candidates = find_candidate_items(userid)
    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    raise NotImplementedError()
    ###########################################################################
    #                            END OF YOUR CODE                             #
    ###########################################################################

    return r_list

In [ ]:
#3. Test the recommended movie
ratings = generate_utility_matrix()
test_case = [0,100,850]
items_testing = [[317, 356, 473, 650, 654], [14, 257, 272, 312, 1015], [95, 97, 99, 172, 180]]
r_movies = []
for i in test_case:
  toplist = user2userRecommendation(ratings,i,5)
  r_movies.append(toplist)

for item in r_movies:
  if sorted(item) in items_testing:
    items_testing.remove(sorted(item))
if len(items_testing)==0:
  print("YOU PASSED THE TEST!")
else:
  print("DOUBLE CHECK YOUR CALCULATION!")